In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/pjd/lib/RSoXS_tutorial/fiberRSA/')
from fiberRSA.fiberRSA import create_all_fiber, upscale_all_fibers, dilate_all_par, dilate_all_par_hollow, find_angles

In [55]:
def Rz(angle):
    return np.array([[np.cos(angle), -np.sin(angle), 0],
                     [np.sin(angle), np.cos(angle), 0],
                     [0, 0, 1]])

def Ry(angle):
    return np.array([[np.cos(angle), 0, np.sin(angle)],
                    [0, 1, 0],
                    [-np.sin(angle), 0, np.cos(angle)]])


def create_R(theta, psi):
    rz = Rz(psi)
    ry = Ry(theta)
    return rz@ry


def create_uni_tensor(delta_ord, beta_ord, delta_ex, beta_ex):
    uni_tensor = np.zeros((delta_ord.shape[0], 3,3), dtype=complex)
    for i in range(len(uni_tensor)):
        uni_tensor[i, 0, 0] = complex(1-delta_ord[i], beta_ord[i])
        uni_tensor[i, 1, 1] = complex(1-delta_ord[i], beta_ord[i])
        uni_tensor[i, 2, 2] = complex(1-delta_ex[i], beta_ex[i])
    return uni_tensor

def create_uni_tensor2(nrss_opts):
    # [dPara, bPara, dPerp, bPerp]
    uni_tensor = np.zeros((len(nrss_opts), 3, 3), dtype=complex)
    for i, key in enumerate(nrss_opts.keys()):
        uni_tensor[i, 0, 0] = complex(1-nrss_opts[key][2], nrss_opts[key][3])
        uni_tensor[i, 1, 1] = complex(1-nrss_opts[key][2], nrss_opts[key][3])
        uni_tensor[i, 2, 2] = complex(1-nrss_opts[key][0], nrss_opts[key][1])
    return uni_tensor

def calculate_abs(R, dielectric, E, tfilm, wvl):
    nrot = R@dielectric@R.T
    p = 1/4/np.pi*(nrot@nrot - np.identity(3))@E
    beta_val = 2*np.pi*np.abs(p.imag)@E
    abs_val = np.exp(-4*np.pi/wvl*beta_val*tfilm)
    return abs_val


def calculate_abs_dist(dist, dielectric, E, tfilm, wvl):
    H, x_edges, y_edges = dist
    x_centers = np.diff(x_edges) + x_edges[:-1]
    y_centers = np.diff(y_edges) + y_edges[:-1]
    abs_vals = np.zeros(len(wvl))
    for ii, theta in enumerate(x_centers):
        for jj, psi in enumerate(y_centers):
            R = create_R(theta, psi)
            abs_vals += calculate_abs(R, dielectric, E, tfilm, wvl)*H[ii, jj]
    return abs_vals/np.sum(dist[0])

E = np.array([1,0,0])

In [6]:
BoxXY = 1024
BoxZ = 128
fibers = create_all_fiber(1000, 2.25, .275, np.pi/2, 1e-3, 75, 300, BoxXY, BoxZ, 1)
upscale_all_fibers(fibers, BoxXY, BoxZ)


fibers_dilated = np.zeros((BoxZ, BoxXY, BoxXY))
S = fibers_dilated.copy()
dilate_all_par_hollow(fibers, fibers_dilated, BoxXY, BoxZ, 0.5, inside_value=0)

/home/pjd/lib/RSoXS_tutorial/fiberRSA/fiberRSA.py:198: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  r = np.linspace(-fibers[i].length/2, fibers[i].length/2, int(fibers[i].length+1))


In [ ]:
count=0
# log normal
radius_mu = 2.225
radius_sigma = 0.23
length_lower=75
length_upper=300
# gaussian
# radius_mu = 4.5*2
# radius_sigma=3
hollow_fraction = .325
theta_mu = np.pi/2
theta_sigma = 1/2/np.pi
BoxXY = 2048
BoxZ = 256
inside_value = 0
S_inside = 0


num_repeats = 3

all_fibers = []
scattering_chi_all = np.zeros((55,360,725))
global_dist = np.zeros((20,20))
            
for repeat in range(num_repeats):
    fibers = create_all_fiber(20000, radius_mu, radius_sigma, theta_mu, theta_sigma, length_lower, length_upper, BoxXY, BoxZ, 1)
    upscale_all_fibers(fibers, BoxXY, BoxZ)
    
    
    fibers_dilated = np.zeros((BoxZ, BoxXY, BoxXY))
    S = fibers_dilated.copy()
    dilate_all_par_hollow(fibers, fibers_dilated, BoxXY, BoxZ, hollow_fraction, inside_value=inside_value)
    dilate_all_par_hollow(fibers, S, BoxXY, BoxZ, hollow_fraction, inside_value=S_inside)
    
    fibers_dilated = downscale_local_mean(fibers_dilated, (2,2,2))
    S = downscale_local_mean(S, (2,2,2))
    theta, psi = find_angles(fibers_dilated)
    theta_np = cp.asnumpy(theta)
    psi_np = cp.asnumpy(psi)
    
    dist1 = np.histogram2d(theta_np[fibers_dilated != 0], psi_np[fibers_dilated != 0], bins=20, density=True,range=[[0,np.pi],[-np.pi, np.pi]])
    global_dist += dist1[0]
    ### NRSS ###
    vacuum_space = 1 - fibers_dilated
    
    mat1 = Material(Vfrac=fibers_dilated, S=S, theta=theta_np, psi=psi_np, energies=energies, name='HOPG')
    mat1.opt_constants = HOPG_opts.opt_constants
    
    Svac = np.zeros(vacuum_space.shape)
    
    mat2 = Material(Vfrac=vacuum_space, S=Svac, theta=Svac, psi=Svac, energies=energies, name='vacuum')
    
    config = {'CaseType':0, 'MorphologyType':0, 'WindowingType':1, 'Energies':energies, 'EAngleRotation':[0.0, 1.0, 360.0]}
    morph = Morphology(2, {1:mat1, 2:mat2}, PhysSize=1, config=config)
    
    scattering = morph.run(stdout=False, stderr=False)
    scattering['qx'] = scattering.qx/10
    scattering['qy'] = scattering.qy/10
    scattering_chi = integrator.integrateImageStack(scattering)
    scattering_chi_all += scattering_chi.values
    for fiber in fibers:
        all_fibers.append(np.round(fiber.radius))

    scattering_chi_all /= num_repeats
    scattering_chi_all = xr.DataArray(scattering_chi_all, dims=['energy', 'chi', 'q'], coords={'energy':scattering_chi.energy, 'chi':scattering_chi.chi, 'q':scattering_chi.q})
    scattering_chi_all.attrs['radius_mu'] = radius_mu
    scattering_chi_all.attrs['radius_sigma'] = radius_sigma
    scattering_chi_all.attrs['hollow_fraction'] = hollow_fraction
    scattering_chi_all.attrs['fiber_radii'] = all_fibers
    # scattering_chi_all.to_netcdf(f'parameter_sweep/data/paramsweep_{count}.nc')

global_abs = calculate_abs_dist([global_distj, dist1[1], dist1[2]],mwcnt_uni,E,100,wvl)
_ = save_plot(scattering_chi_all*global_abs[:,np.newaxis,np.newaxis], radius_mu, radius_sigma, hollow_fraction, count, show=True)